In [1]:
# Analytics Expression Language Example

import xarray as xr
from datacube.api import API
from datacube.ndexpr import NDexpr

In [2]:
# Instantiating API and NDexpr
g = API()
nd = NDexpr()

In [3]:
# construct data request parameters for band_30 and band_40
data_request_descriptor = {
    'platform': 'LANDSAT 5',
    'product': 'NBAR',
    'variables': ('band_30', 'band_40'),
    'dimensions': {
        'longitude': {
            'range': (150, 150.256)
        },
        'latitude': {
            'range': (-34.0, -33.744)
        }
    }
}

# Retrieving data from API
d1 = g.get_data(data_request_descriptor)

# construct data request parameters for PQ
pq_request_descriptor = {
    'platform': 'LANDSAT 5',
    'product': 'PQ',
    'variables': ('band_pixelquality'),
    'dimensions': {
        'longitude': {
            'range': (150, 150.256)
        },
        'latitude': {
            'range': (-34.0, -33.744)
        }
    }
}

# Retrieving data from API
d2 = g.get_data(pq_request_descriptor)

In [4]:
# The following 3 lines shouldn't be done like this
# Currently done like this for the sake of the example.
b30 = xr.DataArray(d1['arrays']['band_30'])
b40 = xr.DataArray(d1['arrays']['band_40'])
pq = nd.get_pqa_mask(d2['arrays']['band_pixelquality'].values)

In [5]:
# NDexpr demo begins here

# perform ndvi as expressed in this language.
ndvi = nd.evaluate('((b40 - b30) / (b40 + b30))')

# perform mask on ndvi as expressed in this language.
masked_ndvi = nd.evaluate('ndvi{pq}')


In [6]:
print(ndvi)

<xarray.DataArray (time: 13, latitude: 1024, longitude: 1024)>
array([[[-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        ..., 
        [ 0.73297491,  0.69905106,  0.68346774, ...,  0.70918822,
          0.73099762,  0.73852004],
        [ 0.71465517,  0.7000449 ,  0.67726626, ...,  0.73426573,
          0.73890032,  0.75965167],
        [ 0.70223979,  0.69293478,  0.7090739 , ...,  0.75767803,
          0.74681934,  0.75966184]],

       [[-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ],
   

In [7]:
print(masked_ndvi)

<xarray.DataArray (time: 13, latitude: 1024, longitude: 1024)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ..., 
        [ 0.73297491,  0.69905106,  0.68346774, ...,  0.70918822,
          0.73099762,  0.73852004],
        [ 0.71465517,  0.7000449 ,  0.67726626, ...,  0.73426573,
          0.73890032,  0.75965167],
        [ 0.70223979,  0.69293478,  0.7090739 , ...,  0.75767803,
          0.74681934,  0.75966184]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
   

In [8]:
# currently dimensions are integer indices, later will be labels when
# Data Access API Interface has been finalised.
reduction_on_dim0 = nd.evaluate('median(masked_ndvi, 0)')
reduction_on_dim01 = nd.evaluate('median(masked_ndvi, 0, 1)')
reduction_on_dim012 = nd.evaluate('median(masked_ndvi, 0, 1, 2)')
print(reduction_on_dim0)
print(reduction_on_dim01)
print(reduction_on_dim012)


<xarray.DataArray (latitude: 1024, longitude: 1024)>
array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ..., 
       [ 0.69706952,  0.65603755,  0.6275599 , ...,  0.70209158,
         0.67934612,  0.66921431],
       [ 0.698231  ,  0.66784134,  0.65029994, ...,  0.73426573,
         0.6961256 ,  0.6974359 ],
       [ 0.68662283,  0.68048757,  0.68241104, ...,  0.74228395,
         0.73004021,  0.7053455 ]])
Coordinates:
  * latitude   (latitude) float64 -33.74 -33.74 -33.74 -33.74 -33.75 -33.75 ...
  * longitude  (longitude) float64 150.0 150.0 150.0 150.0 150.0 150.0 150.0 ...
<xarray.DataArray (longitude: 1024)>
array([ 0.6722499 ,  0.67778837,  0.67570912, ...,  0.73339048,
        0.73039648,  0.73055214])
Coordinates:
  * longitu